In [16]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [0]:
import time
from math import sqrt

In [0]:
class DisjointSetUnion:
    def __init__(self, size):
        self.parent_ = list(range(size))
        self.ranks_ = [0] * size

    def find(self, node):
        if self.parent_[node] != node:
            self.parent_[node] = self.find(self.parent_[node])
        return self.parent_[node]

    def union_sets(self, first, second):
        f_root = self.find(first)
        s_root = self.find(second)
        if f_root == s_root:
            return False
        if self.ranks_[f_root] < self.ranks_[s_root]:
            self.parent_[f_root] = s_root
        elif self.ranks_[f_root] > self.ranks_[s_root]:
            self.parent_[s_root] = f_root
        else:
            self.parent_[s_root] = f_root
            self.ranks_[f_root] += 1
        return True

    def result(self):
        return list(map(self.find, self.parent_))

In [0]:
def distance_1(title1, title2, num_of_words):
    # simple vec distance
    id1 = 0
    id2 = 0
    vec = []
    while id1 < len(title1) and id2 < len(title2):
        x = y = 0
        word1 = title1[id1][0]
        word2 = title2[id2][0]
        if word1 <= word2:
            x = title1[id1][1]
            id1 += 1
        if word1 >= word2:
            y = title2[id2][1]
            id2 += 1
        vec.append(abs(x - y))
    while id1 < len(title1):
        vec.append(title1[id1][1])
        id1 += 1
    while id2 < len(title2):
        vec.append(title2[id2][1])
        id2 += 1
    return sqrt(sum(map(lambda x: x * x, vec)))

In [0]:
def distance_2(title1, title2, num_of_words):
    # manhattan vec distance
    id1 = 0
    id2 = 0
    vec = []
    while id1 < len(title1) and id2 < len(title2):
        x = y = 0
        word1 = title1[id1][0]
        word2 = title2[id2][0]
        if word1 <= word2:
            x = title1[id1][1]
            id1 += 1
        if word1 >= word2:
            y = title2[id2][1]
            id2 += 1
        vec.append(abs(x - y))
    while id1 < len(title1):
        vec.append(title1[id1][1])
        id1 += 1
    while id2 < len(title2):
        vec.append(title2[id2][1])
        id2 += 1
    return sum(vec)

In [0]:
def distance_3(title1, title2, num_of_words):
    # Pearson
    x = [0] * num_of_words
    for elem in title1:
        x[elem[0] - 1] += elem[1]
    y = [0] * num_of_words
    for elem in title2:
        y[elem[0] - 1] += elem[1]
    x_m = sum(x) / len(x)
    y_m = sum(y) / len(y)
    x = [x[_] - x_m for _ in range(len(x))]
    y = [y[_] - y_m for _ in range(len(x))]
    a = sum([x[_] * y[_] for _ in range(len(x))])
    b = sum([x[_] * x[_] for _ in range(len(x))])
    c = sum([y[_] * y[_] for _ in range(len(x))])
    if a == 0:
        return 1
    return 1 - (a / sqrt(b * c))

In [0]:
MAX_INT = 2147483647

In [0]:
def cluster(ver, eds, num_of_clusters, dist, num_words):
    penalty = MAX_INT
    # return DisjointSetUnion of edges
    res = DisjointSetUnion(len(ver))
    # Your code here
    eds.sort()
    k = len(ver)
    for ed in eds:
        u, v = ed[1], ed[2]
        if res.find(u) != res.find(v):
            k -= 1
            res.union_sets(u, v)
        if k == num_of_clusters:
            break
    for ed in eds:
      u, v = ed[1], ed[2]
      if res.find(u) != res.find(v):
        penalty = ed[0]
        break
    return res, penalty

In [0]:
import random
def cluster_2(ver, eds, num_of_clusters, dist, num_words):
    res = DisjointSetUnion(len(ver))
    penalty = 0
    center = random.randint(0, len(ver) - 1)
    id_centers = set()
    id_centers.add(center)
    
    for i in range(num_of_clusters - 1):
        temp_max = 0
        temp_id = 0
        for cen in id_centers:
            for id_v, v in enumerate(ver):
                if id_v not in id_centers:
                    d = dist(v, ver[cen], num_words)
                    if d > temp_max:
                        temp_max = d
                        temp_id = id_v
        id_centers.add(temp_id)
    
    penalty = 0
    
    for v in range(len(ver)):
        if v not in id_centers:
            temp_min = MAX_INT
            temp_id = v
            for u in id_centers:
                d = dist(ver[v], ver[u], num_words)
                if temp_min > d:
                    temp_min = d
                    temp_id = u
            res.union_sets(v, temp_id)
    eds.sort(reverse=True)
    for ed in eds:
      if res.find(ed[1]) == res.find(ed[2]):
        penalty = ed[0]
        break
      
    return res, penalty

In [0]:
def process(filename, cluster, distance, clusters_num, min_count, max_count):
    t_start = time.time()

    # input file
#    filename = 'test_50.txt'

    # function of distance
#    distance = dist

    # clusters num
#    clusters_num = 15

    # word freq param
#   min_count = min_c
#    max_count = max_c

    vertices = []
    with open('/content/drive/My Drive/AISD_labs/' + filename) as f:
        doc_num = int(f.readline())
        words_num = int(f.readline())
        lines_num = int(f.readline())
        words_count = [0] * words_num
        vertices = [[] for _ in range(doc_num)]
        for i in range(lines_num):
            docID, wordID, count = map(int, f.readline().split())
            words_count[wordID - 1] += count
            if count <= max_count:
                vertices[docID - 1].append([wordID, count])

    new_v = []
    for elem in vertices:
        new_v.append([])
        for k in elem:
            if min_count <= words_count[k[0] - 1] <= max_count:
                new_v[-1].append(k)
        if len(new_v[-1]) == 0:
            print("Warning: vertex without words")
    vertices = new_v
    print('Vertices:', len(vertices))

    edges = [(distance(vertices[i], vertices[j], words_num), i, j)
             for i in range(len(vertices) - 1) for j in range(i + 1, len(vertices))]
    print('Edges:', len(edges))

    components, penalty = cluster(vertices, edges, clusters_num, distance, words_num)

    comp = components.result()
    d = dict()
    for i in range(len(comp)):
        if comp[i] not in d:
            d[comp[i]] = [i + 1]
        else:
            d[comp[i]].append(i + 1)
    for elem in d:
        print("Cluster", elem, ":", len(d[elem]), "elems", ":", d[elem])
    print("Penalty =", penalty)
    print("Time:", time.time() - t_start)






# Протестируем на 50 вершинах кластеризацию на основе минимального остовного дерева, максимизирующую минимальное межкластерное расстояние;


In [0]:
process('test_50.txt', cluster, distance_1, clusters_num = 5, min_count = 2, max_count = 10)

Vertices: 50
Edges: 1225
Cluster 3 : 45 elems : [1, 2, 3, 4, 5, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 23, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 41, 42, 43, 44, 45, 46, 47, 49, 50]
Cluster 5 : 1 elems : [6]
Cluster 21 : 2 elems : [22, 24]
Cluster 39 : 1 elems : [40]
Cluster 47 : 1 elems : [48]
Penalty = 23.473389188611005
Time: 0.13773512840270996


# на 400 вершинах

In [0]:
process('test_400.txt', cluster, distance_1,  clusters_num = 15, min_count = 2, max_count = 4)

Vertices: 400
Edges: 79800
Cluster 29 : 379 elems : [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 155, 156, 157, 158, 159, 160, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221,

## На 1000 вершинах

In [0]:
process('test_1000.txt', cluster, distance_1,  clusters_num = 20, min_count = 0, max_count = 100)

Vertices: 1000
Edges: 499500
Cluster 268 : 981 elems : [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 124, 125, 126, 127, 128, 129, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 21

# Тестируем с помощью манхеттонской метрики

In [0]:
process('test_50.txt', cluster, distance_2, 25, 2, 10)

Vertices: 50
Edges: 1225
Cluster 0 : 1 elems : [1]
Cluster 1 : 1 elems : [2]
Cluster 3 : 25 elems : [3, 4, 5, 8, 10, 15, 17, 19, 21, 23, 25, 27, 28, 29, 30, 32, 33, 34, 35, 36, 37, 38, 39, 42, 46]
Cluster 5 : 1 elems : [6]
Cluster 6 : 1 elems : [7]
Cluster 8 : 1 elems : [9]
Cluster 10 : 1 elems : [11]
Cluster 11 : 1 elems : [12]
Cluster 12 : 1 elems : [13]
Cluster 13 : 1 elems : [14]
Cluster 15 : 1 elems : [16]
Cluster 17 : 1 elems : [18]
Cluster 19 : 1 elems : [20]
Cluster 21 : 2 elems : [22, 24]
Cluster 25 : 1 elems : [26]
Cluster 30 : 1 elems : [31]
Cluster 39 : 1 elems : [40]
Cluster 40 : 1 elems : [41]
Cluster 42 : 1 elems : [43]
Cluster 43 : 1 elems : [44]
Cluster 44 : 1 elems : [45]
Cluster 46 : 1 elems : [47]
Cluster 47 : 1 elems : [48]
Cluster 48 : 1 elems : [49]
Cluster 49 : 1 elems : [50]
Penalty = 167
Time: 0.12240314483642578


In [0]:
process('test_400.txt', cluster, distance_2, 50, 2, 10)

Vertices: 400
Edges: 79800
Cluster 29 : 325 elems : [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 51, 52, 53, 55, 56, 57, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 109, 110, 111, 112, 113, 115, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 129, 130, 131, 132, 133, 134, 135, 136, 137, 139, 140, 141, 143, 145, 147, 148, 151, 152, 155, 158, 166, 168, 170, 171, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 193, 194, 195, 196, 197, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 225, 226, 227, 228, 229, 230, 231, 232, 235, 236, 237, 238, 239, 240, 241, 243, 244, 245, 246, 247, 248, 

In [0]:
process('test_1000.txt', cluster, distance_2, 50, 2, 10)

Vertices: 1000
Edges: 499500
Cluster 268 : 941 elems : [1, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 105, 106, 107, 109, 110, 111, 112, 114, 115, 117, 118, 120, 121, 124, 125, 126, 127, 128, 129, 131, 132, 133, 134, 139, 140, 141, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 155, 156, 157, 158, 159, 160, 161, 162, 164, 165, 167, 168, 169, 170, 171, 172, 173, 174, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 2

# C помощью 3-ей метрики

In [0]:
process('test_50.txt', cluster, distance_3, 15, 2, 10)

Vertices: 50
Edges: 1225
Cluster 5 : 26 elems : [1, 6, 7, 8, 11, 12, 13, 18, 19, 20, 23, 25, 26, 29, 30, 31, 32, 33, 35, 36, 39, 40, 41, 44, 45, 48]
Cluster 1 : 1 elems : [2]
Cluster 2 : 1 elems : [3]
Cluster 3 : 2 elems : [4, 5]
Cluster 8 : 1 elems : [9]
Cluster 9 : 1 elems : [10]
Cluster 13 : 3 elems : [14, 43, 50]
Cluster 14 : 4 elems : [15, 16, 17, 21]
Cluster 21 : 2 elems : [22, 24]
Cluster 26 : 2 elems : [27, 28]
Cluster 33 : 3 elems : [34, 46, 49]
Cluster 36 : 1 elems : [37]
Cluster 37 : 1 elems : [38]
Cluster 41 : 1 elems : [42]
Cluster 46 : 1 elems : [47]
Penalty = 0.9392844560108129
Time: 2.770094633102417


In [0]:
process('test_400.txt', cluster, distance_3, 15, 2, 10)

Vertices: 400
Edges: 79800
Cluster 111 : 380 elems : [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 73, 74, 75, 76, 77, 78, 79, 80, 81, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 98, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 123, 124, 125, 126, 127, 129, 131, 132, 133, 134, 135, 136, 137, 138, 139, 141, 142, 144, 145, 147, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 2

# Кластеризация, приближенно минимизирующая максимальное внутрикластерное расстояние.#


## Евклидова метрика

In [0]:
process('test_50.txt', cluster_2, distance_1, 5, 2, 10)

Vertices: 50
Edges: 1225
Cluster 0 : 46 elems : [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 23, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 41, 42, 43, 44, 45, 46, 47, 49, 50]
Cluster 21 : 1 elems : [22]
Cluster 23 : 1 elems : [24]
Cluster 39 : 1 elems : [40]
Cluster 47 : 1 elems : [48]
Penalty = 32.66496594212215
Time: 0.22289204597473145


In [0]:
process('test_400.txt', cluster_2, distance_1, 10, 2, 10)

Vertices: 400
Edges: 79800
Cluster 1 : 388 elems : [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 115, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 

## На 1000 вершинах

In [17]:
process('test_1000.txt', cluster_2, distance_1, 25, 2, 10)

Vertices: 1000
Edges: 499500
Cluster 0 : 976 elems : [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 124, 125, 126, 127, 128, 129, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218

## Манхеттанская метрика

In [0]:
process('test_50.txt', cluster_2, distance_2, 5, 2, 10)

Vertices: 50
Edges: 1225
Cluster 0 : 46 elems : [1, 2, 3, 4, 5, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 23, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50]
Cluster 5 : 1 elems : [6]
Cluster 21 : 1 elems : [22]
Cluster 23 : 1 elems : [24]
Cluster 39 : 1 elems : [40]
Penalty = 495
Time: 0.19565677642822266


In [0]:
process('test_400.txt', cluster_2, distance_2, 15, 2, 10)

Vertices: 400
Edges: 79800
Cluster 0 : 386 elems : [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 115, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 155, 156, 157, 158, 159, 160, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 

## На 1000 вершинах

In [11]:
process('test_1000.txt', cluster_2, distance_2, 30, 2, 10)

Vertices: 1000
Edges: 499500
Cluster 0 : 953 elems : [1, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 42, 43, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 105, 106, 107, 108, 109, 110, 111, 112, 114, 115, 116, 117, 118, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 131, 132, 133, 134, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 22

## С помощью 3-ей метрики

In [0]:
process('test_50.txt', cluster_2, distance_3, 10, 2, 10)

Vertices: 50
Edges: 1225
Cluster 0 : 10 elems : [1, 2, 6, 11, 12, 18, 23, 25, 26, 36]
Cluster 2 : 8 elems : [3, 13, 19, 30, 32, 33, 35, 41]
Cluster 20 : 3 elems : [4, 21, 29]
Cluster 4 : 1 elems : [5]
Cluster 6 : 4 elems : [7, 17, 27, 28]
Cluster 7 : 8 elems : [8, 14, 15, 38, 42, 43, 46, 50]
Cluster 8 : 10 elems : [9, 10, 20, 34, 37, 40, 44, 45, 48, 49]
Cluster 15 : 4 elems : [16, 22, 31, 39]
Cluster 23 : 1 elems : [24]
Cluster 46 : 1 elems : [47]
Penalty = 1.011549677267829
Time: 7.947933912277222


In [0]:
process('test_400.txt', cluster_2, distance_3, 15, 2, 10)

Vertices: 400
Edges: 79800
Cluster 0 : 59 elems : [1, 8, 9, 22, 24, 25, 32, 37, 40, 42, 54, 55, 62, 71, 77, 84, 86, 87, 96, 106, 118, 121, 125, 131, 151, 155, 158, 159, 181, 186, 190, 193, 196, 202, 206, 211, 220, 229, 230, 245, 249, 260, 269, 276, 289, 298, 307, 308, 311, 313, 314, 315, 370, 371, 372, 373, 383, 385, 397]
Cluster 1 : 31 elems : [2, 3, 18, 44, 57, 60, 132, 139, 145, 192, 203, 217, 222, 228, 257, 262, 263, 279, 280, 309, 339, 340, 341, 342, 343, 344, 345, 346, 347, 386, 394]
Cluster 3 : 53 elems : [4, 15, 16, 27, 29, 35, 43, 48, 56, 59, 61, 65, 68, 78, 90, 94, 104, 105, 129, 141, 152, 153, 154, 182, 187, 191, 197, 200, 201, 204, 215, 219, 225, 240, 252, 254, 270, 277, 285, 303, 304, 305, 312, 317, 318, 325, 327, 352, 353, 368, 377, 381, 389]
Cluster 4 : 32 elems : [5, 20, 36, 72, 74, 95, 100, 101, 112, 115, 130, 167, 169, 207, 223, 224, 233, 234, 239, 267, 271, 281, 295, 299, 310, 322, 323, 330, 338, 350, 354, 357]
Cluster 5 : 66 elems : [6, 7, 12, 13, 17, 21, 28, 38, 45

## На 1000 вершинах

In [12]:
process('test_1000.txt', cluster_2, distance_3, 50, 2, 10)

Vertices: 1000
Edges: 499500


KeyboardInterrupt: ignored